In [1]:
# !wget https://huggingface.co/datasets/TokenBender/glaive_coder_raw_text/resolve/main/glaive_dataset_recreated.parquet

In [2]:
import pandas as pd
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai3-aiservices-1652986261.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-35-turbo'

In [4]:
df = pd.read_parquet('glaive_dataset_recreated.parquet')

In [5]:
instructions = df['instruction'].tolist()

In [6]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [7]:
prompt = f'terjemah ke bahasa melayu: {instructions[0]}'

In [8]:
!mkdir glaive_coder_raw_text

mkdir: cannot create directory ‘glaive_coder_raw_text’: File exists


In [9]:
def generate(instruction, index):
    filename = f'glaive_coder_raw_text/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        prompt = f'terjemah ke bahasa melayu: {instruction}'
        ins = predict(prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [10]:
generate(instructions[0], 0)

In [11]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range((len(instructions) // 6) * 0, (len(instructions) // 6) * 1, max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

 46%|████████████████▉                    | 5206/11342 [00:13<00:14, 435.98it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 77%|███████████████████████████▋        | 8736/11342 [00:14<00:02, 1055.43it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


100%|████████████████████████████████████| 11342/11342 [00:16<00:00, 689.42it/s]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
unsupported operand type(s) for +: 'NoneType' and 'str'
